In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Data Quality Check") \
    .getOrCreate()

In [ ]:
df = spark.read.csv('../../data/bank.csv', header=True, inferSchema=True)
df.printSchema()

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']
df.select(numeric_features).describe().toPandas().transpose()

In [ ]:
# df.describe().toPandas().transpose()

# Check for duplicates

In [ ]:
duplicate_rows = df.count() - df.dropDuplicates().count()
print(f"Number of duplicate rows: {duplicate_rows}")

# Check for unique values in columns

In [ ]:
for column in df.columns:
    print(f"{column}: {df.select(column).distinct().count()} distinct values")

# Check for missing values

In [ ]:
from pyspark.sql.functions import col, sum

df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).toPandas()

'''
sum(col(c).isNull().cast("int")):
    For each column c, the expression checks if the values are null using col(c).isNull().
    The cast("int") converts the Boolean result (True or False) into an integer (1 for True and 0 for False).
    The sum function adds up all the 1s (representing null values) for each column, giving the total count of null values for that column.

alias(c): 
    The result for each column is given an alias with the column name c.

select(*( ... )):
    The * unpacks the list of expressions generated by the comprehension for each column, allowing select to take multiple column expressions.

toPandas():
    The result is converted from a PySpark DataFrame to a Pandas DataFrame, making it easier to work with in Python, particularly if you need to perform further analysis or visualization.
'''

# Categorical value distributions

In [ ]:
df.groupBy("education").count().toPandas()

# Find outliers

In [ ]:
quantiles = df.approxQuantile("age", [0.01, 0.5, 0.99], 0.0)
print(quantiles)

lower_bound = quantiles[0]
median = quantiles[1]
upper_bound = quantiles[2]

outliers = df.filter((df["age"] < lower_bound) | (df["age"] > upper_bound))
outliers.toPandas()

# Correlation

In [ ]:
from pyspark.sql.functions import corr

df.select(corr("age", "balance")).toPandas()

In [ ]:
# Assuming df has a column 'email'
# email_pattern = "^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
# invalid_emails = df.filter(~F.col("email").rlike(email_pattern))